citc: rl

blaze-bin/experimental/users/ppiech/rl/notebook.par > /dev/null 2>&1 & disown

In [2]:
from __future__ import absolute_import, division, print_function

from absl import logging
import sys
import time
import os
import subprocess
import base64
import imageio
import ffmpeg
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
#import reverb
import random

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
import tf_agents.environments.utils 
from tf_agents.eval import metric_utils
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import greedy_policy
from tf_agents.policies import policy_saver
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from spaceship_env import SpaceshipEnv

#from colabtools import adhoc_import

# with adhoc_import.Google3CitcClient('rl'):
#   # Now we can import pure-python code from anywhere in google3, including experimental.
#   #
#   # NOTE: anything with a (transitive) build-requiring dependency such as C++
#   # code, protocol buffers, etc. that isn't already built in to the colab_binary
#   # requires a build_targets= param. See go/adhoc-build below.
#   from google3.pyglib import gfile
#   from google3.experimental.users.ppiech.spaceship_learn.spaceship_env import SpaceshipEnv

In [3]:
def pipe_to_ffmpeg(filename):
  args = (
        ffmpeg
        .input('pipe:')
        .output(filename)
        .overwrite_output()
        .compile()
    )
  return subprocess.Popen(args, stdin=subprocess.PIPE)

def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

In [4]:
num_iterations = 40000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"}
collect_steps_per_iteration = 10 # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 5000  # @param {type:"integer"}

log_interval = 1000  # @param {type:"integer"}
train_checkpoint_interval=10000 # @param {type:"integer"}
policy_checkpoint_interval=5000 # @param {type:"integer"}
rb_checkpoint_interval=2000 # @param {type:"integer"}
keep_rb_checkpoint=False # @param {type:"boolean"}
train_sequence_length=1 # @param {type:"integer"}
summary_interval=1000 # @param {type:"integer"}
summaries_flush_secs=10 # @param {type:"integer"}

In [5]:
root_dir = os.path.expanduser('log')
train_dir = os.path.join(root_dir, 'train')
eval_dir = os.path.join(root_dir, 'eval')
saved_model_dir = os.path.join(root_dir, 'policy_saved_model')
if not tf.io.gfile.exists(saved_model_dir):
  tf.io.gfile.makedirs(saved_model_dir)

In [6]:
def test_env():
  # env_name = 'CartPole-v0'
  # env = suite_gym.load(env_name)

  env = SpaceshipEnv()
  tf_agents.environments.utils.validate_py_environment(env, episodes=5)

#test_env()

env = SpaceshipEnv()


/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/specs/array_spec.py:352: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == -np.inf] = low
/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/specs/array_spec.py:353: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == np.inf] = high


In [7]:
def snapshot_render():
  env.reset()
  print(env.time_step_spec())
  PIL.Image.fromarray(env.render())

#snapshot_render()

In [8]:
def print_env_specs():
  print('Observation Spec:')
  print(env.time_step_spec().observation)
  print('Reward Spec:')
  print(env.time_step_spec().reward)
  print('Action Spec:')
  print(env.action_spec())

#print_env_specs()

In [9]:
def capture_random_video():
  video_filename = '/tmp/starship_env_random.mp4'

  video_pipe = pipe_to_ffmpeg(video_filename)

  env.reset()
  for _ in range(100):
    env.step(random.randint(-1, 1))
    image = PIL.Image.fromarray(env.render())
    image.save(video_pipe.stdin, 'BMP')

  video_pipe.stdin.close()
  video_pipe.wait()

  embed_mp4(video_filename)

#capture_random_video()

In [10]:
def print_time_step():
  time_step = env.reset()
  print('Time step:')
  print(time_step)

  action = np.array(1, dtype=np.int32)

  next_time_step = env.step(action)
  print('Next time step:')
  print(next_time_step)

#print_time_step()

In [11]:
# train_py_env = suite_gym.load(env_name)
# eval_py_env = suite_gym.load(env_name)
train_py_env = SpaceshipEnv()
eval_py_env = SpaceshipEnv()

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [22]:
#
# Policy
#
fc_layer_params = (256, 256)
action_tensor_spec = tensor_spec.from_spec(train_py_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [26]:
#
# Agent
#
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0, dtype=tf.int64)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

policy_checkpointer = common.Checkpointer(
    ckpt_dir=os.path.join(train_dir, 'policy'),
    max_to_keep=None,
    policy=agent.policy,
    global_step=agent.train_step_counter)

saved_model = policy_saver.PolicySaver(
    agent.policy)

def save_policy(global_step_value):
    """Saves policy using both checkpoint saver and saved model."""
    policy_checkpointer.save(global_step=global_step_value)
    saved_model_path = os.path.join(
        saved_model_dir, 'policy_' + ('%d' % global_step_value).zfill(8))
    saved_model.save(saved_model_path)

eval_policy = agent.policy
collect_policy = agent.collect_policy

In [27]:
#
# Replay buffer
#

table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

# table = reverb.Table(
#     table_name,
#     max_size=replay_buffer_max_length,
#     sampler=reverb.selectors.Uniform(),
#     remover=reverb.selectors.Fifo(),
#     rate_limiter=reverb.rate_limiters.MinSize(1),
#     signature=replay_buffer_signature)

# reverb_server = reverb.Server([table])

# replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
#     agent.collect_data_spec,
#     table_name=table_name,
#     sequence_length=2,
#     local_server=reverb_server)

# rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
#   replay_buffer.py_client,  #not py_client
#   table_name,
#   sequence_length=2)

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec, 
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

print('RB capacity: %i' % replay_buffer.capacity)
rb_observer = replay_buffer.add_batch

rb_ckpt_dir = os.path.join(train_dir, 'replay_buffer')
rb_checkpointer = common.Checkpointer(
    ckpt_dir=rb_ckpt_dir, max_to_keep=1, replay_buffer=replay_buffer)


RB capacity: 100000


In [28]:
#
# Train Metrics
#

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(batch_size=train_env.batch_size),
    tf_metrics.AverageEpisodeLengthMetric(batch_size=train_env.batch_size),
    tf_metrics.ChosenActionHistogram(),
]

train_checkpointer = common.Checkpointer(
    ckpt_dir=train_dir,
    max_to_keep=1,
    agent=agent,
    global_step=agent.train_step_counter,
    metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))


In [29]:
#
# Driver
#
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec(), 
                                                info_spec=agent.collect_policy.info_spec)

random_policy = agent.collect_policy

agent_observers = [replay_buffer.add_batch]

DynamicStepDriver(
    train_env, agent.collect_policy, [rb_observer], num_steps=1000).run()
# driver.num_steps = 1000
# driver.run()

# py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       random_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=initial_collect_steps).run(train_py_env.reset())

# # Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

iterator = iter(dataset)

In [30]:
#
# Eval Metrics
#

eval_summary_writer = tf.compat.v2.summary.create_file_writer(
    eval_dir, 
    flush_millis=summaries_flush_secs * 1000
)

eval_metrics = [
    tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
    tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes),
]

eval_policy =  greedy_policy.GreedyPolicy(agent.policy)
 
metric_utils.eager_compute(
    eval_metrics,
    eval_env,
    eval_policy,
    num_episodes=num_eval_episodes,
    summary_writer=eval_summary_writer,
    train_step=0
)

metric_utils.log_metrics(eval_metrics)

In [18]:
#
# Train Loop
#

try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Reset the environment.
time_step = train_env.reset()

collect_driver = DynamicStepDriver(
    train_env, agent.collect_policy, [rb_observer], 
    num_steps=collect_steps_per_iteration)

timed_at_step = agent.train_step_counter.numpy()
time_acc = 0

for _ in range(num_iterations):
  start_time = time.time()

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step_tf = agent.train_step_counter
  step = step_tf.numpy()

  time_acc += time.time() - start_time

  for train_metric in train_metrics:
    train_metric.tf_summaries(
        train_step=agent.train_step_counter, step_metrics=train_metrics[:2])

  if step % train_checkpoint_interval == 0:
    train_checkpointer.save(global_step=step)

  if step % policy_checkpoint_interval == 0:
    save_policy(step)

  if step % rb_checkpoint_interval == 0:
    rb_checkpointer.save(global_step=step)

  if step % log_interval == 0:
    print('step = %d, loss = %f' % (step, train_loss.numpy()))
    steps_per_sec = (step- timed_at_step) * collect_steps_per_iteration / time_acc
    print('%.3f steps/sec' % steps_per_sec)
    tf.compat.v2.summary.scalar(
        name='env_steps_per_sec', data=steps_per_sec, step=step_tf)
    timed_at_step = step
    time_acc = 0

  if step % eval_interval == 0:
    metric_utils.eager_compute(
      eval_metrics,
      eval_env,
      eval_policy,
      num_episodes=num_eval_episodes,
      summary_writer=eval_summary_writer,
      train_step=step
    )

    metric_utils.log_metrics(eval_metrics)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 1000, loss = 5870.727051
132.804 steps/sec


/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/utils/common.py:1539: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []
/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/utils/common.py:1539: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []
/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if policy_state is ():  # pylint: disable=literal-comparison
/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if policy_state is ():  # pylint: disable=literal-comparison
/home/ppiech/git/spaceship_learn/env/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:258: SyntaxW

KeyboardInterrupt: 

In [31]:
metric_utils.eager_compute(
      eval_metrics,
      eval_env,
      eval_policy,
      num_episodes=num_eval_episodes,
      summary_writer=eval_summary_writer,
      train_step=step
    )

print (eval_metrics)

[<tf_agents.metrics.tf_metrics.AverageReturnMetric object at 0x7f11083fbd60>, <tf_agents.metrics.tf_metrics.AverageEpisodeLengthMetric object at 0x7f11b9beab90>]


In [32]:
train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(batch_size=train_env.batch_size),
    tf_metrics.AverageEpisodeLengthMetric(batch_size=train_env.batch_size),
    tf_metrics.ChosenActionHistogram(),
]

train_checkpointer = common.Checkpointer(
    ckpt_dir=train_dir,
    max_to_keep=1,
    agent=agent,
    global_step=agent.train_step_counter,
    metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))



In [33]:
%load_ext google3.learning.brain.tensorboard.notebook.extension

ModuleNotFoundError: No module named 'google3'

In [ ]:
from google3.third_party.tensorboard import notebook
notebook.list()

In [ ]:
%tensorboard --logdir="log" --port=0

In [34]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

[swscaler @ 0x5d57d00] Warning: data is not aligned! This can lead to a speed loss
